In [14]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # Last point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

# Define posture evaluation functions
def assess_barbell_curl(landmarks):
    left_elbow_angle = calculate_angle(
        [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
        [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
        [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
    )
    if 50 <= left_elbow_angle <= 150:
        return "Correct posture!"
    return "Incorrect elbow angle. Keep the motion controlled."

def assess_deadlift(landmarks):
    back_angle = calculate_angle(
        [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
        [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
        [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
    )
    if 160 >= back_angle <= 180:
        return "Correct posture!"
    return "Incorrect back alignment. Maintain a straight back."

def assess_squat(landmarks):
    knee_angle = calculate_angle(
        [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
        [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
        [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
    )
    if 90 >= knee_angle <= 120:
        return "Correct posture!"
    return "Incorrect squat depth. Ensure proper angle of leg joints ."

def assess_lateral_raises(landmarks):
    shoulder_angle = calculate_angle(
        [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
        [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
        [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
    )
    if 80 <= shoulder_angle <= 100:
        return "Correct posture!"
    return "Incorrect shoulder angle. Lift arms parallel to the floor."

def assess_overhead_press(landmarks):
    elbow_angle = calculate_angle(
        [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
        [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
        [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
    )
    if 160 >= elbow_angle <= 180:
        return "Correct posture!"
    return "Incorrect elbow form. Fully extend arms overhead."

# Create a directory to save images
output_dir = "correct_posture_images"
os.makedirs(output_dir, exist_ok=True)

# Video capture (0 for webcam)
file_dir = './unknown/OverheadPressDumbbell.MOV'
cap = cv2.VideoCapture(file_dir)

# Exercise selection
exercise = "OverheadPress"  # Change to "BarbellCurl", "Deadlift", etc., as needed
feedback = "Analyzing posture..."

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()

    # Recolor the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make detection
    results = pose.process(image)

    # Recolor the image back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Create a copy of the frame without annotations
    raw_frame = frame.copy()

    # Extract landmarks
    try:
        landmarks = results.pose_landmarks.landmark

        # Select the appropriate assessment function
        if exercise == "BarbellCurl":
            feedback = assess_barbell_curl(landmarks)
        elif exercise == "Deadlift":
            feedback = assess_deadlift(landmarks)
        elif exercise == "Squat":
            feedback = assess_squat(landmarks)
        elif exercise == "LateralRaises":
            feedback = assess_lateral_raises(landmarks)
        elif exercise == "OverheadPress":
            feedback = assess_overhead_press(landmarks)

        # Render landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Display the feedback on the annotated image
        cv2.putText(image, feedback, (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 3.1, (0, 255, 0) if "Correct" in feedback else (0, 0, 255), 5, cv2.LINE_AA)

        # Combine raw frame and annotated frame
        combined_frame = np.hstack((raw_frame, image))

        # Save combined frame if posture is correct
        if feedback: # Add "Correct before feedback to take only the correct"
            combined_filename = os.path.join(output_dir, f"combined_frame_{frame_count}.jpg")
            cv2.imwrite(combined_filename, combined_frame)

    except Exception as e:
        feedback = "Landmarks not detected."
        combined_frame = frame.copy()  # Use the raw frame if landmarks are not detected

    # Display the video feed with and without annotations
    cv2.imshow('Weight Training Assessment', combined_frame)

    frame_count += 1

    # Break the loop with 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1734016101.356929   80860 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1734016101.374119   99802 gl_context.cc:357] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.0.9-0ubuntu0.2), renderer: D3D12 (NVIDIA GeForce RTX 3070)
W0000 00:00:1734016101.524653   99792 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734016101.586816   99789 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
